## Connect to AWS S3

In [1]:
# Create Spark Session

from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Connect AWS") \
    .setMaster("local[*]") \

conf.set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.3.2")

spark = SparkSession \
    .builder \
    .config(conf=conf) \
    .getOrCreate()

spark

:: loading settings :: url = jar:file:/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-58952316-3fb1-4107-bd40-6d595b38f19c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.1026 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 187ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.1026 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.2 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------

In [8]:
# Read file from S3
df = spark \
    .read \
    .format("csv") \
    .option("header", True) \
    .load("s3a://easewithdata/dw-with-pyspark/landing/store/*.csv")

df.show()

+--------+--------------+--------------+--------------+-----+--------+------------+
|store_id|    store_name|       address|          city|state|zip_code|phone_number|
+--------+--------------+--------------+--------------+-----+--------+------------+
|    S000|Not Applicable|Not Applicable|Not Applicable|   NA|   00000|000-000-0000|
|    S001|  Pet House TX|   123 Main St|       Anytown|   TX|   12345|555-555-5555|
|    S002|  Pet House NY|    456 Elm St|   Anothertown|   NY|   67890|555-555-5556|
|    S003|  Pet House CA|   789 Oak Ave|       Bigcity|   CA|   09876|555-555-5557|
|    S004|  Pet House FL|321 Birch Blvd|    Smallville|   FL|   55555|555-555-5558|
|    S005|  Pet House IL|   654 Pine St|      Busytown|   IL|   11111|555-555-5559|
|    S006|  Pet House PA|  987 Cedar Rd|    Quietville|   PA|   22222|555-555-5560|
|    S007|  Pet House AZ|  246 Maple St|      Anywhere|   AZ|   33333|555-555-5561|
+--------+--------------+--------------+--------------+-----+--------+------

In [9]:
# Write the data to AWS S3

df.coalesce(1) \
    .write \
    .format("parquet") \
    .save("s3a://easewithdata/dw-with-pyspark/target/demo/store_parquet")    